# 라이브러리 불러오기

In [1]:
# ! pip install -U imbalanced-learn scikit-learn

In [2]:
# ! pip install catboost

In [3]:
import numpy as np
import pandas as pd
import re
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

from nltk.tokenize import TreebankWordTokenizer # 표준 토큰화
from nltk.corpus import stopwords # 불용어 제거
from nltk.stem import WordNetLemmatizer # 기본 형태로 변환
from imblearn.over_sampling import SMOTE # 비대칭이라 사용

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# 데이터 불러오기

In [4]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
submission_df = pd.read_csv('./data/sample_submission.csv')

In [5]:
display(train_df.head())
display(test_df.head())

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."


In [6]:
train_df.groupby('first_party_winner').size()

first_party_winner
0     829
1    1649
dtype: int64

# 데이터 전처리

In [7]:
# 문자 처리
cat_cols = ['first_party', 'second_party', 'facts']

# \b : 단어 경계, W* : 길이가 0이상이고 단어가 아닌 문자, w{1} : 길이가 1인 단어
short_word = re.compile(r'\W*\b\w{1}\b') # 길이가 1인 단어 찾기
tokenizer = TreebankWordTokenizer() # 단어 단위로 토큰화
stopword = stopwords.words('english') # 불용어 리스트 가져오기
lemmatizer = WordNetLemmatizer() # 단어의 기본 형태 가져오기

In [8]:
vec = CountVectorizer(ngram_range = (1, 2)) # 출현빈도
vec_facts = TfidfVectorizer(ngram_range = (1, 2)) # 단어토큰

## 단어 처리

In [9]:
def prepro1(df, cols, short_word, tokenizer, stopword, lemmatizer):
    first_party_lst = []
    second_party_lst = []
    facts_lst = []
    
    for col in cols:
        df[col] = df[col].str.strip() # 공백 제거
        df[col] = df[col].str.lower() # 소문자로 변경
        df[col] = df[col].str.replace(',', '')
        df[col] = df[col].str.replace('.', '')
        
        if col == 'first_party':
            for content in df[col]:
                content = short_word.sub('', content) # 한 글자 단어 제거
                com = re.compile(r"[^\uAC00-\uD7A30-9a-zA-Z\s]") # 한글, 영어, 숫자 및 공백 문자를 제외한 모든 문자를 매칭
                content = com.sub('', content)
                tokens = tokenizer.tokenize(content) # 단어 토큰화
                token_lst = []
                for token in tokens:
                    if token not in stopword: #불용어 제거
                        token_lst.append(lemmatizer.lemmatize(token, 'n')) # 단어의 기본 형태 가져오기
                first_party_lst.append(token_lst)
            # 단어들 결합
            for i in range(len(first_party_lst)):
                first_party_lst[i] = ' '.join(first_party_lst[i])
                
        elif col == 'second_party':
            for content in df[col]:
                content = short_word.sub('', content) # 한 글자 단어 제거
                com = re.compile(r"[^\uAC00-\uD7A30-9a-zA-Z\s]") # 한글, 영어, 숫자 및 공백 문자를 제외한 모든 문자를 매칭
                content = com.sub('', content)
                tokens = tokenizer.tokenize(content) # 단어 토큰화
                token_lst = []
                for token in tokens:
                    if token not in stopword: #불용어 제거
                        token_lst.append(lemmatizer.lemmatize(token, 'n')) # 단어의 기본 형태 가져오기
                second_party_lst.append(token_lst)
            # 단어들 결합
            for i in range(len(second_party_lst)):
                second_party_lst[i] = ' '.join(second_party_lst[i])
                
        elif col == 'facts':
            for content in df[col]:
                content = short_word.sub('', content) # 한 글자 단어 제거
                com = re.compile(r"[^\uAC00-\uD7A30-9a-zA-Z\s]") # 한글, 영어, 숫자 및 공백 문자를 제외한 모든 문자를 매칭
                content = com.sub('', content)
                tokens = tokenizer.tokenize(content) # 단어 토큰화
                token_lst = []
                for token in tokens:
                    if token not in stopword: #불용어 제거
                        token_lst.append(lemmatizer.lemmatize(token, 'n')) # 단어의 기본 형태 가져오기
                facts_lst.append(token_lst)
            # 단어들 결합
            for i in range(len(facts_lst)):
                facts_lst[i] = ' '.join(facts_lst[i])
                
    return first_party_lst, second_party_lst, facts_lst

## 벡터화

In [10]:
def prepro2(first, second, facts, vec, vec_facts, is_train):
    if is_train:
        vec.fit(first + second) # conut
        vec_facts.fit(facts) # Tf
    
    X_first = vec.transform(first).toarray()
    X_second = vec.transform(first).toarray()
    X_facts = vec_facts.transform(facts).toarray()
    
    return np.concatenate([X_first, X_second, X_facts], axis = 1)

전처리 실행

In [11]:
train_first, train_second, train_facts = prepro1(train_df, cat_cols, short_word, tokenizer, stopword, lemmatizer)
test_first, test_second, test_facts = prepro1(test_df, cat_cols, short_word, tokenizer, stopword, lemmatizer)

In [12]:
X_train = prepro2(train_first, train_second, train_facts, vec, vec_facts, True)
y_train = train_df['first_party_winner']
X_test = prepro2(test_first, test_second, test_facts, vec, vec_facts, False)

In [13]:
print('Train shape : {}, {}'.format(X_train.shape, y_train.shape))
print('Test shape : {}'.format(X_test.shape))

Train shape : (2478, 204261), (2478,)
Test shape : (1240, 204261)


## 불균형 데이터 처리

SMOTE

- 소수 클래스의 샘플을 합성하여 데이터셋을 오버 샘플링하는 방법으로, 데이터 불균형 문제를 해결하는 데 널리 사용되는 기법

In [14]:
X_smote, y_smote = smote = SMOTE(k_neighbors=5).fit_resample(X_train, y_train)
print('train shape : {}, {}'.format(X_smote.shape, y_smote.shape))

train shape : (3298, 204261), (3298,)


In [15]:
y_smote.value_counts()

first_party_winner
1    1649
0    1649
Name: count, dtype: int64

# 모델링

## 데이터 분리

train데이터 셋으로 train, val 데이터 셋으로 분리

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_smote, y_smote, random_state = 42, test_size = 0.2, 
                                                 stratify = y_smote)

print('Train shape : {}, {}'.format(X_train.shape, y_train.shape))
print(y_train.value_counts())

print()

print('Validation shape : {}, {}'.format(X_val.shape, y_val.shape))
print(y_val.value_counts())

Train shape : (2638, 204261), (2638,)
first_party_winner
1    1319
0    1319
Name: count, dtype: int64

Validation shape : (660, 204261), (660,)
first_party_winner
0    330
1    330
Name: count, dtype: int64


In [17]:
model_lst = ['Logistic Regression', 'KNN Classifier', 'Decision Tree', 'Random Forest', 'Ridge Claffifier',
            'XGBoost', 'AdaBoost', 'Catboost']

In [18]:
models = [ LogisticRegression(max_iter = 500, random_state = 123),
                   KNeighborsClassifier(n_neighbors = 149, n_jobs = -1),
                   DecisionTreeClassifier(),
                   RandomForestClassifier(n_estimators = 100),
                   RidgeClassifier(),
                   XGBClassifier(),
                   AdaBoostClassifier(),
                   CatBoostClassifier(verbose = 0, random_state = 113)]

In [19]:
def score(X_train, y_train, X_val, y_val, names, models):
    score_df = pd.DataFrame()
    score_train_lst = []
    score_val_lst = []
    
    for model in models:
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)
        score_train_lst.append(accuracy_score(y_train, y_train_pred))
        score_val_lst.append(accuracy_score(y_val, y_val_pred))
        
    score_df['model'] = names
    score_df['Train accuracy'] = score_train_lst
    score_df['Validation accuracy'] = score_val_lst
    score_df = score_df.sort_values('Validation accuracy', ascending = False)
    return score_df
        

In [20]:
score(X_train, y_train, X_val, y_val, model_lst, models)

C:\Users\mit005\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


,model,Train accuracy,Validation accuracy
3,Random Forest,1.000000,0.824242
4,Ridge Claffifier,0.999621,0.821212
0,Logistic Regression,0.990902,0.809091
5,XGBoost,0.995072,0.766667
7,Catboost,0.965883,0.765152
2,Decision Tree,1.000000,0.703030
6,AdaBoost,0.757013,0.684848
1,KNN Classifier,0.509856,0.518182


# 제출

In [21]:
y_pred  = best_logistic.predict(X_test)
submission_df['first_party_winner'] = y_pred
# submission_df.to_csv('neighbourhoodcleaningrule_logi.csv', index = False)

NameError: name 'best_logistic' is not defined